<a href="https://colab.research.google.com/github/hosngholami/RecommenderSystem/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import warnings
warnings.filterwarnings('ignore')

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
import pandas as pd
import numpy as np
import random
from sklearn.neighbors import NearestNeighbors

pd.set_option("display.max_columns", 100)
pd.set_option('display.max_rows', 100)

In [98]:
hotel = pd.read_csv('/content/drive/MyDrive/Shiraz/hotel.csv')
historic =  pd.read_csv('/content/drive/MyDrive/Shiraz/historic.csv')
museum = pd.read_csv('/content/drive/MyDrive/Shiraz/museum.csv')
park = pd.read_csv('/content/drive/MyDrive/Shiraz/park.csv')
restaurant = pd.read_csv('/content/drive/MyDrive/Shiraz/restaurant.csv')
hospital = pd.read_csv('/content/drive/MyDrive/Shiraz/hospital.csv')
placeOfWorship = pd.read_csv('/content/drive/MyDrive/Shiraz/place_of_worship.csv')
swimmingPool = pd.read_csv('/content/drive/MyDrive/Shiraz/swimming_pool.csv')
caffe = pd.read_csv('/content/drive/MyDrive/Shiraz/caffe.csv')

In [104]:
placeOfWorship

,id,@id,addr:city,addr:street,addr:town,air_conditioning,alt_name,amenity,baby_feeding,building,building_2,check_date,denomination,description,heritage,heritage:operator,historic,landuse,layer,level,loc_name,name,name:ar,name:de,name:en,name:fa,name:it,name:ru,name:zh,opening_hours,place_of_worship:type,ref:ichhto,religion,service_times,subject:wikidata,subject:wikipedia,tomb,tourism,type,wheelchair,wikidata,wikipedia,@geometry,geometry
0,relation/1082540,relation/1082540,NaN,NaN,NaN,NaN,NaN,place_of_worship,NaN,mosque,NaN,NaN,shia,NaN,2.0,ichhto,yes,NaN,NaN,NaN,NaN,مسجد وکیل,NaN,Vakil Moschee,Vakil Mosque,NaN,vakil mosque,Мечеть Вакиль,NaN,NaN,NaN,182.0,muslim,NaN,NaN,NaN,NaN,NaN,multipolygon,NaN,Q1831907,fa:مسجد وکیل,center,POINT (52.5453684 29.6142619)
1,relation/2982490,relation/2982490,NaN,NaN,NaN,NaN,NaN,place_of_worship,NaN,yes,NaN,NaN,NaN,NaN,2.0,ichhto,yes,NaN,NaN,NaN,NaN,مسجد نصیرالملک,NaN,Nasir-al-Molk-Moschee,Nasir al-Mulk Mosque,مسجد نصیرالملک,NaN,NaN,莫克粉紅清真寺,NaN,NaN,396.0,muslim,NaN,NaN,NaN,NaN,attraction,multipolygon,NaN,Q1962312,fa:مسجد نصیرالملک,center,POINT (52.5484996 29.6085101)
2,relation/15514605,relation/15514605,NaN,NaN,NaN,NaN,NaN,place_of_worship,NaN,yes,NaN,NaN,shia,NaN,2.0,ichhto,yes,NaN,NaN,NaN,NaN,مسجد حاج علی,NaN,NaN,Haj Ali Mosque,NaN,NaN,NaN,NaN,NaN,NaN,4514.0,muslim,NaN,NaN,NaN,NaN,NaN,multipolygon,NaN,Q5881316,fa:مسجد حاج علی,center,POINT (52.547786 29.6105593)
3,relation/15650641,relation/15650641,NaN,NaN,NaN,NaN,NaN,place_of_worship,NaN,mosque,NaN,NaN,shia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,مسجد حاج میرزا کریم,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,muslim,NaN,NaN,NaN,NaN,NaN,multipolygon,NaN,NaN,NaN,center,POINT (52.5476344 29.6081047)
4,relation/15853089,relation/15853089,NaN,NaN,NaN,NaN,NaN,place_of_worship,NaN,mosque,NaN,NaN,shia,NaN,2.0,ichhto,yes,NaN,NaN,NaN,NaN,مسجد امامزاده سید تاج الدین,NaN,NaN,Imamzadeh Taj al-Din Gharib,NaN,NaN,NaN,NaN,NaN,NaN,4513.0,muslim,NaN,NaN,NaN,NaN,NaN,multipolygon,limited,Q5930332,fa:بقعه سید تاج‌الدین غریب,center,POINT (52.5342595 29.6088531)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,node/11215293820,node/11215293820,NaN,NaN,NaN,NaN,NaN,place_of_worship,NaN,NaN,NaN,2023-09-24,shia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,مسجد فیضیه,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,muslim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (52.5559427 29.6320301)
212,node/11224801791,node/11224801791,NaN,NaN,NaN,NaN,NaN,place_of_worship,NaN,NaN,NaN,2023-09-28,shia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,muslim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (52.5495764 29.6033111)
213,node/11229722253,node/11229722253,NaN,NaN,NaN,NaN,NaN,place_of_worship,NaN,NaN,NaN,2023-09-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,muslim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (52.5350131 29.6072781)
214,node/11293045219,node/11293045219,NaN,NaN,NaN,NaN,NaN,place_of_worship,NaN,NaN,NaN,2023-10-27,shia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,مسجد و حسینیه قائم آل محمد,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,muslim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (52.4878255 29.5709593)


In [58]:
hotel["type"] = "hotel"
historic["type"] = "historic"
museum["type"] = "museum"
park["type"] = "park"
restaurant["type"] = "restaurant"
hospital["type"] = "hospital"
placeOfWorship["type"] = "placeofworship"
swimmingPool["type"] = "swimmingpool"
caffe["type"] = "caffe"

In [60]:
museum = museum[["amenity", "building", "heritage", "historic", "museum", "tourism", "geometry"]]
museum.to_csv('/content/drive/MyDrive/Shiraz/museum.csv')

In [61]:
park = park[["leisure", "name", "name:en", "opening_hours", "geometry"]]
park.to_csv('/content/drive/MyDrive/Shiraz/park.csv')

In [62]:
restaurant = restaurant[["amenity", "branch", "building", "cuisine", "name", "name:en", "opening_hours", "tourism", "geometry"]]
restaurant.to_csv('/content/drive/MyDrive/Shiraz/restaurant.csv')

In [63]:
hospital = hospital[["amenity",
          "name",
          "name:en",
          "opening_hours",
          "geometry"]]

hospital.to_csv('/content/drive/MyDrive/Shiraz/hospital.csv')

In [64]:
placeOfWorship = placeOfWorship[["amenity",
                "historic",
                "name",
                "name:en",
                "opening_hours",
                "place_of_worship:type",
                "tourism",
                "religion",
                "geometry"]]

placeOfWorship.to_csv('/content/drive/MyDrive/Shiraz/placeOfWorship.csv')

In [70]:
locations = pd.concat([historic,
                       museum,
                       park,
                       restaurant,
                       placeOfWorship,
                       swimmingPool,
                       caffe])
locations["Latitude"] = 0
locations["Longitude"] = 0

In [71]:
len(locations)

1223

In [76]:
for i in range(len(locations)):
  location = locations.iloc[i]['geometry']
  latitude = str(location).split(" ")[1]
  latitude = latitude.replace("'", "")
  latitude = latitude.replace("(", "")

  longitude = location.split(" ")[2]
  longitude = longitude.replace("'", "")
  longitude = longitude.replace(")", "")
  longitude = longitude.replace("(", "")
  longitude = longitude.replace(",", "")


  locations.iloc[i]['Latitude'] = latitude
  locations.iloc[i]['Longitude'] = longitude






In [77]:
print(len(locations))

1223


In [78]:
locations.to_csv('/content/drive/MyDrive/Shiraz/locations.csv')

In [79]:
locations = pd.read_csv('/content/drive/MyDrive/Shiraz/locations.csv')

In [80]:
len(locations)

1223

In [21]:
tag = pd.DataFrame(columns=["Id", "title:en", "title:fn"])

In [ ]:
data = [
{
    "Id": 1,
    "Title": "Historical",
    "Title:Fn" : "مکان‌های تاریخی مانند آثار باستانی و معماری قدیمی."
},{
    "Id": 2,
    "Title": "Recreational",
    "Title:Fn" : "مکان‌های تفریحی مانند پارک‌ها، شهربازی‌ها و مراکز خرید."
},{
    "Id": 3,
    "Title": "Natural",
    "Title:Fn" : "جاذبه‌های طبیعی مانند کوه‌ها، جنگل‌ها، دریاها و آبشارها."
},{
    "Id": 4,
    "Title": "Adventure",
    "Title:Fn" : "مکان‌های ماجراجویانه (مانند کوه‌نوردی، صخره‌نوردی)."
},{
    "Id": 5,
    "Title": "Religious",
    "Title:Fn" : "مکان‌های مذهبی (مانند کلیساها، مساجد، معابد)."
},{
    "Id": 6,
    "Title": "Health/Thermal",
    "Title:Fn" : "مکان‌های سلامت و درمانی مانند چشمه‌های آب گرم"
}, {
    "Id": 7,
    "Title": "Resort",
    "Title:Fn" : "استراحتگاه."
},
{
    "Id": 8,
    "Title": "Artistic",
    "Title:Fn" : "مکان‌هایی هنری مانند گالری‌ها، موزه‌ها و مناظر زیبا."
},
{
    "Id": 9,
    "Title": "Photography",
    "Title:Fn" : " مکان های مناسب عکاسی مانند گالری، موزه ها و مناظر زیبا."
},
{
    "Id": 10,
    "Title": "Cultural",
    "Title:Fn" : "مکان‌هایی که فرهنگ و هنر یک منطقه را نمایش می‌دهند."
},
]

In [22]:
tag = pd.read_csv('/content/drive/MyDrive/Shiraz/tag.csv')

In [23]:
tag

,Unnamed: 0,Id,Title,Title:Fn
0,0,1,Historical,مکان‌های تاریخی مانند آثار باستانی و معماری قد...
1,1,2,Recreational,مکان‌های تفریحی مانند پارک‌ها، شهربازی‌ها و مر...
2,2,3,Natural,جاذبه‌های طبیعی مانند کوه‌ها، جنگل‌ها، دریاها ...
3,3,4,Adventure,مکان‌های ماجراجویانه (مانند کوه‌نوردی، صخره‌نو...
4,4,5,Religious,مکان‌های مذهبی (مانند کلیساها، مساجد، معابد).
5,5,6,Health/Thermal,مکان‌های سلامت و درمانی مانند چشمه‌های آب گرم
6,6,7,Resort,استراحتگاه.
7,7,8,Artistic,مکان‌هایی هنری مانند گالری‌ها، موزه‌ها و مناظر...
8,8,9,Photography,مکان های مناسب عکاسی مانند گالری، موزه ها و م...
9,9,10,Cultural,مکان‌هایی که فرهنگ و هنر یک منطقه را نمایش می‌...


In [81]:
# locations[locations["type"] == "historic"]['tag'] = "historic"
locations["tag"] = [[] for _ in range(len(locations))]

print(set(locations["amenity"]))
for i in range(len(locations)):
  if (locations.iloc[i]["amenity"] == 'cafe'):
    if('caffe' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('caffe')

  if (locations.iloc[i]["amenity"] == 'restaurant'):
    if('restaurant' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('restaurant')

  if (locations.iloc[i]["amenity"] == 'theatre'):
    if('Artistic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Artistic')
    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')


  if (locations.iloc[i]["amenity"] == 'place_of_worship'):
    if('Religious' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Religious')

print(len(locations))


{'public_bath', 'place_of_worship', 'restaurant', 'marketplace', 'cafe', nan, 'theatre'}
1223


In [82]:
filtered_df = locations[locations['tag'].apply(lambda x: 'Cultural' in x)]
filtered_df["tag"]

,tag
92,"[Artistic, Cultural]"


In [83]:
set(locations["building"])

for i in range(len(locations)):
  if (locations.iloc[i]["building"] == 'mosque'):
    if('Religious' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Religious')

  if (locations.iloc[i]["building"] == 'mausoleum'):
    if('Religious' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Religious')
    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

  if (locations.iloc[i]["building"] == 'sports_hall'):
    if('Recreational' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Recreational')

print(len(locations))

1223


In [84]:
print(type(locations["heritage"]))

for i in range(len(locations)):
  if (locations.iloc[i]["heritage"] == 2):
    if('Religious' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Religious')

  if (locations.iloc[i]["building"] == 'mausoleum'):
    if('Religious' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Religious')
    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

  if (locations.iloc[i]["building"] == 'sports_hall'):
    if('Recreational' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Recreational')
print(len(locations))

<class 'pandas.core.series.Series'>
1223


In [85]:
print(set(locations["historic"]))
for i in range(len(locations)):
  if (locations.iloc[i]["historic"] == 'caravanserai'):
    if('historic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('historic')

    if('Photography' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Photography')

    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

  if (locations.iloc[i]["historic"] == 'castle'):
    if('historic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('historic')

    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

    if('Photography' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Photography')

  if (locations.iloc[i]["historic"] == 'castle_wall'):
    if('historic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('historic')

    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

    if('Photography' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Photography')

  if (locations.iloc[i]["historic"] == 'city_gate'):
    if('historic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('historic')

    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

  if (locations.iloc[i]["historic"] == 'garden'):
    if('Recreational' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Recreational')

    if('Photography' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Photography')

  if (locations.iloc[i]["historic"] == 'manor'):

    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

    if('Photography' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Photography')

  if (locations.iloc[i]["historic"] == 'monument'):

    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

    if('historic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('historic')

    if('Photography' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Photography')

  if (locations.iloc[i]["historic"] == 'pillory'):

    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

    if('Photography' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Photography')

  if (locations.iloc[i]["historic"] == 'yes'):

    if('historic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('historic')


print(len(locations))

{'garden', 'castle_wall', 'building', 'city_gate', 'monument', 'pillory', 'house', 'caravanserai', 'manor', 'تپه_پوستچی', 'tomb', 'castle', 'yes', 'memorial', nan}
1223


In [86]:
set(locations["leisure"])
for i in range(len(locations)):
  if (locations.iloc[i]["leisure"] == 'garden'):
    if('Recreational' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Recreational')
  if (locations.iloc[i]["leisure"] == 'park'):
    if('Recreational' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Recreational')

  if (locations.iloc[i]["leisure"] == 'swimming_pool'):
    if('Recreational' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Recreational')


In [87]:
set(locations["museum"])

for i in range(len(locations)):
  if (locations.iloc[i]["museum"] == 'art'):
    if('historic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('historic')

    if('Artistic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Artistic')

    if('Photography' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Photography')

  if (locations.iloc[i]["museum"] == 'history'):
    if('historic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('historic')

  if (locations.iloc[i]["museum"] == 'music'):
    if('historic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('historic')
    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

  if (locations.iloc[i]["museum"] == 'music;ceramics'):
    if('historic' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('historic')
    if('Cultural' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Cultural')

In [88]:
set(locations["tourism"])
for i in range(len(locations)):
  if (locations.iloc[i]["tourism"] == 'hostel'):
    if('Resort' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Resort')

  if (locations.iloc[i]["tourism"] == 'hotel'):
    if('Resort' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Resort')


In [89]:
set(locations["religion"])
for i in range(len(locations)):
  if (locations.iloc[i]["religion"] == 'muslim'):
    if('Religious' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Religious')


KeyError: 'religion'

In [90]:
set(locations["sport"])

for i in range(len(locations)):
  if (locations.iloc[i]["sport"] == 'billiards'):
    if('Recreational' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Recreational')

  if (locations.iloc[i]["sport"] == 'swimming'):
    if('Recreational' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Recreational')

In [91]:
set(locations["swimming_pool"])
for i in range(len(locations)):
  if (locations.iloc[i]["swimming_pool"] == 'swimming_pool'):
    if('Recreational' not in locations.iloc[i]["tag"]):
      locations.iloc[i]["tag"].append('Recreational')

In [92]:
locations.to_csv('/content/drive/MyDrive/Shiraz/locations.csv')


In [93]:
locations = locations[["name", "name:en", "tag", "type", "Latitude", "Longitude"]]
locations.to_csv('/content/drive/MyDrive/Shiraz/locations.csv')

In [94]:
locations

,name,name:en,tag,type,Latitude,Longitude
0,مسجد وکیل,Vakil Mosque,"[Religious, historic]",NaN,0,0
1,ارگ کریم خان,Citadel of Karim Khan-e Zand,"[historic, Cultural, Photography]",NaN,0,0
2,مدرسه خان,Madraseh-ye Khan College,[historic],NaN,0,0
3,مسجد نصیرالملک,Nasir al-Mulk Mosque,"[Religious, historic]",NaN,0,0
4,خانه زینت الملوک,Zinat ol Molk House,[],NaN,0,0
...,...,...,...,...,...,...
1218,کافه قرار,Gharar Café,[caffe],caffe,0,0
1219,کافه سه کنج,Seh Konj Cafe,[caffe],caffe,0,0
1220,کافه پدربزرگ,Pedarbozorg Cafe,[caffe],caffe,0,0
1221,کافهtohi,NaN,[caffe],caffe,0,0


In [95]:
checkList = {}

for i in range(len(locations)):
   key = str(locations.loc[i, "Latitude"]) + str(locations.loc[i, "Longitude"])

   if(key not in checkList):
     checkList[key] = 0
   else:
     checkList[key] += 1
print(len(checkList.keys()))

1
